In [1]:
import os
import torch
import torch.utils.data
import torchvision
from PIL import Image
from pycocotools.coco import COCO
import cv2
import matplotlib.pyplot as plt
from sklearn.metrics import f1_score, precision_score, recall_score, cohen_kappa_score, classification_report, confusion_matrix
from scipy.stats import rankdata
from numpy import linalg as LA
# import intel_extension_for_pytorch as ipex

%load_ext autotime

# import albumentations as A
# from albumentations.pytorch import ToTensorV2
import numpy as np
import torch.nn.functional as F

# from torchmetrics.detection.mean_ap import MeanAveragePrecision
from torch.optim.lr_scheduler import MultiStepLR
import time
import os
plt.style.use('ggplot')
device = 'cuda'
import os
import time
import math
from tqdm import tqdm
import logging
from torch.autograd import Variable
from torch.utils.tensorboard import SummaryWriter
from torch.optim.lr_scheduler import StepLR, MultiStepLR

seed = 42
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
torch.cuda.manual_seed_all(seed) 

time: 172 ms (started: 2024-05-05 23:35:08 +06:00)


In [2]:
class_dictionary = {1: 'person', 2: 'bike', 3: 'car', 4: 'motor', 6: 'bus',  7: 'train', 8: 'truck', 10: 'light', 
                   11: 'hydrant', 12: 'sign', 17: 'dog', 37: 'skateboard', 73: 'stroller', 77: 'scooter', 79: 'other vehicle'
                   }

time: 0 ns (started: 2024-05-05 23:35:13 +06:00)


In [3]:
##################################################################
#   Utils
##################################################################


"""
Tensorboard logger code referenced from:
https://github.com/yunjey/pytorch-tutorial/blob/master/tutorials/04-utils/
Other helper functions:
https://github.com/cs230-stanford/cs230-stanford.github.io
"""

import json
import logging
import os
import shutil
import torch
from collections import OrderedDict
from torch.optim.lr_scheduler import _LRScheduler
# import tensorflow as tf
import numpy as np
import scipy.misc 
try:
    from StringIO import StringIO  # Python 2.7
except ImportError:
    from io import BytesIO         # Python 3.x


class Params():
    """Class that loads hyperparameters from a json file.

    Example:
    ```
    params = Params(json_path)
    print(params.learning_rate)
    params.learning_rate = 0.5  # change the value of learning_rate in params
    ```
    """

    def __init__(self, json_path):
        with open(json_path) as f:
            params = json.load(f)
            self.__dict__.update(params)

    def save(self, json_path):
        with open(json_path, 'w') as f:
            json.dump(self.__dict__, f, indent=4)
            
    def update(self, json_path):
        """Loads parameters from json file"""
        with open(json_path) as f:
            params = json.load(f)
            self.__dict__.update(params)

    @property
    def dict(self):
        """Gives dict-like access to Params instance by `params.dict['learning_rate']"""
        return self.__dict__


class RunningAverage():
    """A simple class that maintains the running average of a quantity
    
    Example:
    ```
    loss_avg = RunningAverage()
    loss_avg.update(2)
    loss_avg.update(4)
    loss_avg() = 3
    ```
    """
    def __init__(self):
        self.steps = 0
        self.total = 0
    
    def update(self, val):
        self.total += val
        self.steps += 1
    
    def __call__(self):
        return self.total/float(self.steps)

class AverageMeter(object):

    def __init__(self):
        self.reset()

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val*n
        self.count += n
        self.avg = self.sum/self.count

def get_lr(optimizer):
    for param_group in optimizer.param_groups:
        return param_group['lr']

def set_logger(log_path):
    """Set the logger to log info in terminal and file `log_path`.

    In general, it is useful to have a logger so that every output to the terminal is saved
    in a permanent file. Here we save it to `model_dir/train.log`.

    Example:
    ```
    logging.info("Starting training...")
    ```

    Args:
        log_path: (string) where to log
    """
    logger = logging.getLogger()
    logger.setLevel(logging.INFO)

    if not logger.handlers:
        # Logging to a file
        file_handler = logging.FileHandler(log_path)
        file_handler.setFormatter(logging.Formatter('%(asctime)s:%(levelname)s: %(message)s'))
        logger.addHandler(file_handler)

        # Logging to console
        stream_handler = logging.StreamHandler()
        stream_handler.setFormatter(logging.Formatter('%(message)s'))
        logger.addHandler(stream_handler)


def save_dict_to_json(d, json_path):
    """Saves dict of floats in json file

    Args:
        d: (dict) of float-castable values (np.float, int, float, etc.)
        json_path: (string) path to json file
    """
    with open(json_path, 'w') as f:
        # We need to convert the values to float for json (it doesn't accept np.array, np.float, )
        d = {k: float(v) for k, v in d.items()}
        json.dump(d, f, indent=4)


def save_checkpoint(state, is_best, checkpoint, epoch_checkpoint = False):
    """Saves model and training parameters at checkpoint + 'last.pth.tar'. If is_best==True, also saves
    checkpoint + 'best.pth.tar'

    Args:
        state: (dict) contains model's state_dict, may contain other keys such as epoch, optimizer state_dict
        is_best: (bool) True if it is the best model seen till now
        checkpoint: (string) folder where parameters are to be saved
    """
    filepath = os.path.join(checkpoint, 'last.pth.tar')
    if not os.path.exists(checkpoint):
        print("Checkpoint Directory does not exist! Making directory {}".format(checkpoint))
        os.mkdir(checkpoint)
    else:
        print("Checkpoint Directory exists! ")
    torch.save(state, filepath)
    if is_best:
        shutil.copyfile(filepath, os.path.join(checkpoint, 'best.pth.tar'))
    if epoch_checkpoint == True:
        epoch_file = str(state['epoch']-1) + '.pth.tar'
        shutil.copyfile(filepath, os.path.join(checkpoint, epoch_file))





def load_checkpoint(checkpoint, model, optimizer=None):
    """Loads model parameters (state_dict) from file_path. If optimizer is provided, loads state_dict of
    optimizer assuming it is present in checkpoint.

    Args:
        checkpoint: (string) filename which needs to be loaded
        model: (torch.nn.Module) model for which the parameters are loaded
        optimizer: (torch.optim) optional: resume optimizer from checkpoint
    """
    try:

      if not os.path.exists(checkpoint):
        raise FileNotFoundError
    except FileNotFoundError:
      ("File doesn't exist {}".format(checkpoint))

    if torch.cuda.is_available():
        checkpoint = torch.load(checkpoint)
    else:
        # this helps avoid errors when loading single-GPU-trained weights onto CPU-model
        checkpoint = torch.load(checkpoint, map_location=lambda storage, loc: storage)

    model.load_state_dict(checkpoint['state_dict'])

    if optimizer:
        optimizer.load_state_dict(checkpoint['optim_dict'])

    return checkpoint


class WarmUpLR(_LRScheduler):
    """warmup_training learning rate scheduler
    Args:
        optimizer: optimzier(e.g. SGD)
        total_iters: totoal_iters of warmup phase
    """

    def __init__(self, optimizer, total_iters, last_epoch=-1):
        self.total_iters = total_iters
        super().__init__(optimizer, last_epoch)

    def get_lr(self):
        """we will use the first m batches, and set the learning
        rate to base_lr * m / total_iters
        """
        return [base_lr * self.last_epoch / (self.total_iters + 1e-8) for base_lr in self.base_lrs]

time: 16 ms (started: 2024-05-05 23:35:13 +06:00)


In [4]:
transform = {
        'train': torchvision.transforms.Compose([
            torchvision.transforms.Resize([64,64]), # Resizing the image as the VGG only take 224 x 244 as input size
            torchvision.transforms.RandomHorizontalFlip(), # Flip the data horizontally
            #TODO if it is needed, add the random crop
            torchvision.transforms.RandomHorizontalFlip(p=0.5),
            torchvision.transforms.ToTensor(),
#             transforms.Normalize(mean=(0.5,0.5,0.5), std=(0.5,0.5,0.5))
        ]),
        'test': torchvision.transforms.Compose([
            torchvision.transforms.Resize([64,64]),
#             transforms.RandomHorizontalFlip(),
            torchvision.transforms.ToTensor(),
#             transforms.Normalize(mean=(0.5,0.5,0.5), std=(0.5,0.5,0.5))
        ])
    }

time: 0 ns (started: 2024-05-05 23:35:14 +06:00)


In [5]:
train_data_dir = '.\\train_images'
# train_coco = 'images_thermal_train/coco.json'
val_train_dir = '.\\val_images'
# val_coco = 'images_thermal_val/coco.json'
batch_size = 128

train_data = torchvision.datasets.ImageFolder(root=train_data_dir, transform=transform['train'])
train_dataloader_class = torch.utils.data.DataLoader(dataset=train_data, batch_size=batch_size, shuffle=True)

val_data = torchvision.datasets.ImageFolder(root=val_train_dir, transform=transform['test'])
val_dataloader_class = torch.utils.data.DataLoader(dataset=val_data, batch_size=64, shuffle=True)

time: 1.08 s (started: 2024-05-05 23:35:16 +06:00)


In [6]:
from openvino.runtime.ie_api import CompiledModel
from typing import Union


def validate(model, val_loader):
    """Compute the metrics using data from val_loader for the model"""
    total = 0
    correct = 0
    data_sc = []
    data_rc = []
    start_time = time.time()
    # Switch to evaluate mode.
    if not isinstance(model, CompiledModel):
        model.eval()
        model.to(torch_device)

    with torch.no_grad():
        end = time.time()
        for i, (images, labels_batch) in enumerate(val_loader):
            images = images.to('cpu')
            labels_batch = labels_batch.to('cpu')

            # Compute the output.
            if isinstance(model, CompiledModel):
                output_layer = model.output(0)
                output = model(images)[output_layer]
                output_batch = torch.from_numpy(output)
#             print(output)
            loss = 0.0  # force validation loss to zero to reduce computation time
            _, predicted = output_batch.max(1)
            total += labels_batch.size(0)
            correct += predicted.eq(labels_batch).sum().item()
            predicted = predicted.cpu().data.numpy()
            labels_batch = labels_batch.cpu().data.numpy()
            data_sc.extend(predicted)
            data_rc.extend(labels_batch)

    #         break
    #     print(data_rc)
        fscore = f1_score(data_rc, data_sc, average = 'weighted')
        precision = precision_score(data_rc, data_sc, average = 'weighted')
        recall = recall_score(data_rc, data_sc, average = 'weighted')
        kappa = cohen_kappa_score(data_rc, data_sc)
        C = classification_report(data_rc, data_sc)
        matrix = confusion_matrix(data_rc, data_sc)
        accuracy_class = matrix.diagonal()/matrix.sum(axis=1)

        print("Precision: ", precision)
        print("Recall: ", recall)
        print("Report: ", C)
        print("F1: ", fscore)
        print("Kappa: ", kappa)
        print("Matrix: ", matrix)
        print("Accuracy Class: ", accuracy_class)

        acc = 100. * correct / total
        logging.info("- Eval metrics, acc:{acc:.4f}, loss: {loss:.4f}".format(acc=acc, loss=loss))
        my_metric = {'accuracy': acc, 'loss': loss}
        #my_metric['accuracy'] = acc
        return my_metric

time: 47 ms (started: 2024-05-05 23:35:17 +06:00)


In [8]:
def get_model_size(ir_path, m_type: str = "Mb", verbose: bool = True) -> float:
    xml_size = os.path.getsize(ir_path)
    bin_size = os.path.getsize(os.path.splitext(ir_path)[0] + ".bin")
    for t in ["bytes", "Kb", "Mb"]:
        if m_type == t:
            break
        xml_size /= 1024
        bin_size /= 1024
    model_size = xml_size + bin_size
    if verbose:
        print(f"Model graph (xml):   {xml_size:.3f} {m_type}")
        print(f"Model weights (bin): {bin_size:.3f} {m_type}")
        print(f"Model size:          {model_size:.3f} {m_type}")
    return model_size

time: 0 ns (started: 2024-05-02 06:18:54 +06:00)


In [9]:
int8_ir_path = ".\\classifier\\quantized_model.xml"
print(f"[2/7] Save INT8 model: {int8_ir_path}")
int8_model_size = get_model_size(int8_ir_path, verbose=True)

[2/7] Save INT8 model: .\classifier\quantized_model.xml
Model graph (xml):   0.162 Mb
Model weights (bin): 10.692 Mb
Model size:          10.854 Mb
time: 0 ns (started: 2024-05-02 06:18:55 +06:00)


In [11]:
import openvino as ov

core = ov.Core()
model = core.read_model(model=int8_ir_path)
compiled_model = core.compile_model(model=model, device_name='CPU')
validate(compiled_model, val_dataloader_class)

Precision:  0.89324738337159
Recall:  0.895336159903089
Report:                precision    recall  f1-score   support

           0       0.96      0.87      0.91      4465
           1       0.87      0.83      0.85      1945
           2       0.40      0.34      0.37        93
           3       0.79      0.89      0.83      2384
           4       0.67      0.76      0.71       170
           5       0.93      0.98      0.95      7101
           6       0.00      0.00      0.00         3
           7       0.62      0.29      0.40        55
           8       0.58      0.35      0.43       179
           9       0.00      0.00      0.00         6
          10       0.37      0.17      0.24        63
          11       0.33      0.02      0.04        46

    accuracy                           0.90     16510
   macro avg       0.54      0.46      0.48     16510
weighted avg       0.89      0.90      0.89     16510

F1:  0.8918310025007341
Kappa:  0.850858600164687
Matrix:  [[3879   

D:\Downloads\FLIR_ADAS_v2\FLIRA\lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
D:\Downloads\FLIR_ADAS_v2\FLIRA\lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
D:\Downloads\FLIR_ADAS_v2\FLIRA\lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
D:\Downloads\FLIR_A

{'accuracy': 89.5336159903089, 'loss': 0.0}

time: 46.5 s (started: 2024-05-02 07:24:11 +06:00)


In [12]:
validate(compiled_model, val_dataloader_class)

Precision:  0.89324738337159
Recall:  0.895336159903089
Report:                precision    recall  f1-score   support

           0       0.96      0.87      0.91      4465
           1       0.87      0.83      0.85      1945
           2       0.40      0.34      0.37        93
           3       0.79      0.89      0.83      2384
           4       0.67      0.76      0.71       170
           5       0.93      0.98      0.95      7101
           6       0.00      0.00      0.00         3
           7       0.62      0.29      0.40        55
           8       0.58      0.35      0.43       179
           9       0.00      0.00      0.00         6
          10       0.37      0.17      0.24        63
          11       0.33      0.02      0.04        46

    accuracy                           0.90     16510
   macro avg       0.54      0.46      0.48     16510
weighted avg       0.89      0.90      0.89     16510

F1:  0.8918310025007341
Kappa:  0.850858600164687
Matrix:  [[3879   

D:\Downloads\FLIR_ADAS_v2\FLIRA\lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
D:\Downloads\FLIR_ADAS_v2\FLIRA\lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
D:\Downloads\FLIR_ADAS_v2\FLIRA\lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
D:\Downloads\FLIR_A

{'accuracy': 89.5336159903089, 'loss': 0.0}

time: 9.81 s (started: 2024-05-02 07:25:11 +06:00)
